---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx
import pandas
import numpy
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
    """Loads the Employee Movie Choices

    Returns:
     Graph: graph with movie and employees as nodes
    """
    movie_choices = pandas.read_table('Employee_Movie_Choices.txt')
    movie_choices = networkx.from_pandas_dataframe(movie_choices, "#Employee", "Movie")
    return movie_choices        

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    """Adds 'type' to nodes from movie-graph

    Returns:
     Graph: answer_one with 'type' attribute added (employee or movie)
    """
    graph = answer_one()
    new_graph = networkx.Graph()
    nodes = graph.nodes()
    employee_nodes = [node for node in nodes if node in employees]
    movie_nodes = [node for node in nodes if node in movies]
    new_graph.add_nodes_from(employee_nodes, bipartite=0, type='employee')
    new_graph.add_nodes_from(movie_nodes, bipartite=1, type="movie")
    new_graph.add_edges_from(graph.edges())
    return new_graph

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
    graph = answer_two()
    assert networkx.is_bipartite(graph)
    return bipartite.weighted_projected_graph(graph, employees)

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [5]:
def answer_four():
    """calculates the pearson correlation for data

    Returns:
     float: Pearson correlation for weight and relationship_score
    """
    three = answer_three()
    relationships = pandas.read_table(
        "Employee_Relationships.txt",
        names="employee_left employee_right relationship_score".split())
    relationships["employees"] = relationships.apply(
        lambda row: tuple(sorted((row["employee_left"],
                                  row['employee_right']))), axis=1)

    weights = pandas.DataFrame(
        three.edges(data=True),
        columns="employee_left employee_right weight".split())
    weights["weight"] = weights.weight.map(lambda row: row["weight"])
    weights["employees"] = weights.apply(lambda row: tuple(sorted(
        (row["employee_left"],
         row["employee_right"]))),
                                         axis=1)

    joined = pandas.merge(relationships, weights, how="outer", 
                          on=['employees'])
    assert len(joined) == len(relationships)
    joined['weight'] = joined["weight"].fillna(0)

    data = joined[["relationship_score", "weight"]]
    correlation = data.corr()
    return correlation.relationship_score.weight        